In [2]:
import pandas as pd

question_df = pd.read_csv('../data/problem_descriptions.csv')
question_df.head()

,problem_id,problem_description,language
0,p02872,Problem Setting\r\nOverview\r\nSpecification o...,english
1,p01327,Problem G: 天体観測\r\n2012年のある日のこと。\r\n北の田舎町に住むあな...,japanese
2,p02871,Problem Setting\r\nOverview\r\nSpecification o...,english
3,p01482,Memory Leak\r\nTime Limit: 8 sec / Memory Limi...,japanese
4,p01535,Markup language has Declined\r\nE: マークアップ言語は衰退...,japanese


In [3]:
problem_dict = {}
for index, row in question_df.iterrows():
    problem_dict[row['problem_id']] = row['problem_description']

In [16]:
len(problem_dict)

3999

In [84]:
import pandas as pd
q_df = pd.read_csv('../data/502/502_problem_text.csv')
q_df.head()

,problem_id,problem_description
0,45,"Return the sum of the numbers in the array, ex..."
1,48,"Give you an array, write a Java function witho..."
2,51,"Given an array, write a Java function tenRun t..."
3,56,"Given an integer array, write a Java function ..."
4,57,"Given a number n, create and return a new int ..."


In [85]:
question_list = {}
for index, row in q_df.iterrows():
    # question_list[index] = problem_dict[row['id']]
    question_list[index] = row['problem_description']
print(question_list[0])
len(question_list)


Return the sum of the numbers in the array, except ignore sections of numbers starting with a 6 and extending to the next 7 (every 6 will be followed by at least one 7). Return 0 for no numbers.


10

In [42]:
question_list

{0: 'Give you an array of length 3, write a Java function called sum3 to return the sum of the three numbers in the array.',
 1: 'Give you an array, write a Java function called frontPiece, if the array length is greater than or equal to 2, return an array containing the first two elements; otherwise, return the original array.',
 2: 'Give you an array, write a Java function called countEvens to return the number of even numbers.',
 3: 'Give you an array and an int val, write a Java function isEverywhere to check whether, in the array nums, at least one element of every pair of consecutive integers equals val.',
 4: 'Give you an array, write a Java function sum13 that calculates the sum of its elements, but if an element is 13, then that element and the one immediately following it should be excluded from the sum.',
 5: "Given an integer array, write a Java function has77 that checks if the array has two 7's next to each other, or there are two 7's separated by one element. The functio

In [86]:
from transformers import AutoTokenizer, AutoModel
from tqdm import tqdm
import torch
import torch.nn as nn

tokenizer = AutoTokenizer.from_pretrained("../graphcodebert-base")  # 使用的是本地的codebert-base 或 graphcodebert-base
model = AutoModel.from_pretrained("../graphcodebert-base")

new_question_embeddings_768 = []
new_question_embeddings_32 = []
linear_proj = nn.Linear(768, 32)
for idx,question in tqdm(question_list.items()):
    # inputs = tokenizer(skill, return_tensors="pt", padding=True, truncation=False)
    truncated_code = question[:100]
    # 使用 tokenizer 处理数据，并填充到最大长度
    inputs = tokenizer(truncated_code, return_tensors="pt", padding="max_length", max_length=100, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        context_embeddings_768 = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
        context_embeddings_32 = linear_proj(torch.tensor(context_embeddings_768).unsqueeze(0)).squeeze(0).numpy()
        # print(context_embeddings_768)
        # print(context_embeddings_32)
    new_question_embeddings_768.append(context_embeddings_768)
    new_question_embeddings_32.append(context_embeddings_32)

print(len(new_question_embeddings_768))
print(len(new_question_embeddings_32))



Some weights of RobertaModel were not initialized from the model checkpoint at ../graphcodebert-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/10 [00:00<?, ?it/s]/opt/miniconda3/envs/myenv/lib/python3.9/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
100%|██████████| 10/10 [00:00<00:00, 14.77it/s]

10
10


In [19]:
print(len(new_question_embeddings_768))
print(new_question_embeddings_32[0].shape)


10
(32,)


In [87]:
import pickle
with open('../data/502/questions_embeddings_index_768.pkl', 'wb') as f:
    pickle.dump(new_question_embeddings_768, f)
with open('../data/502/questions_embeddings_index_32.pkl', 'wb') as f:
    pickle.dump(new_question_embeddings_32, f)

In [113]:
import pandas as pd
kc_df = pd.read_csv('../data/492/492_kc_list.csv')
kc_df.head()

,kc_id,kc_text
0,1,Loop iteration
1,2,Conditional statements
2,3,String operations
3,4,String substring
4,5,String concatenation


In [114]:
kc_name_dict = {}
for index, row in kc_df.iterrows():
    kc_name_dict[index] = row['kc_text']
len(kc_name_dict)

13

In [115]:
from tqdm import tqdm
import torch
import torch.nn as nn
import pickle
from transformers import AutoTokenizer, AutoModel

tokenizer = AutoTokenizer.from_pretrained("../graphcodebert-base")  # 使用的是本地的codebert-base 或 graphcodebert-base
model = AutoModel.from_pretrained("../graphcodebert-base")

new_kc_embeddings_768 = []
new_kc_embeddings_32 = []
linear_proj = nn.Linear(768, 32)
for idx,kc in tqdm(kc_name_dict.items()):
    # inputs = tokenizer(skill, return_tensors="pt", padding=True, truncation=False)
    truncated_code = kc[:15]
    # 使用 tokenizer 处理数据，并填充到最大长度
    inputs = tokenizer(truncated_code, return_tensors="pt", padding="max_length", max_length=15, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
        context_embeddings_768 = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()
        context_embeddings_32 = linear_proj(torch.tensor(context_embeddings_768).unsqueeze(0)).squeeze(0).numpy()
        # print(context_embeddings_768)
        # print(context_embeddings_32)
    new_kc_embeddings_768.append(context_embeddings_768)
    new_kc_embeddings_32.append(context_embeddings_32)

print(len(new_kc_embeddings_768))
print(len(new_kc_embeddings_32))


with open('../data/492/kc_embeddings_index_768.pkl', 'wb') as f:
    pickle.dump(new_kc_embeddings_768, f)
with open('../data/492/kc_embeddings_index_32.pkl', 'wb') as f:
    pickle.dump(new_kc_embeddings_32, f)

Some weights of RobertaModel were not initialized from the model checkpoint at ../graphcodebert-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  0%|          | 0/13 [00:00<?, ?it/s]/opt/miniconda3/envs/myenv/lib/python3.9/site-packages/torch/nn/modules/module.py:1762: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `RobertaSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
100%|██████████| 13/13 [00:00<00:00, 13.79it/s]

13
13


In [116]:
# 计算知识点的余弦相似度
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
kc_embeddings_768 = torch.tensor(new_kc_embeddings_768)
kc_similarity_768 = cosine_similarity(kc_embeddings_768)
threshold = 0.5
adj_matrix = (kc_similarity_768 > threshold).astype(np.float32)
adj_matrix.shape
np.save('../data/492/adj_kc_codebert_codenet.npy', adj_matrix)
adj_matrix.shape

(13, 13)

In [100]:
adj = np.load('../data/492/adj_kc_codebert_codenet.npy')
adj.shape

(14, 14)

In [26]:
with open('../data/C/questions_embeddings_index_768.pkl', 'rb') as f:
    new_kc_embeddings_768 = pickle.load(f)
print(len(new_kc_embeddings_768))
new_kc_embeddings_768[0].shape

2255


(768,)

In [30]:
with open('../data/C/adj_kc_codebert_codenet.npy', 'rb') as f:
    adj_matrix = np.load(f)
adj_matrix.shape

(73, 73)

In [49]:
def load_dataset_from_pkl(file_path):
    with open(file_path, 'rb') as f:
        dataset = pickle.load(f)
    return dataset

In [101]:
dataset_info['one_hot_matrix'].shape

(10, 13)

In [117]:
import os
import pickle
from processing_codenet import load_dkt_dataset, KTDataset, pad_collate, preprocess_train_dataset
saved_data_dir = "../data/492-2"
dataset_info = load_dataset_from_pkl(os.path.join(saved_data_dir, 'dataset_info.pkl'))
dataset_info['one_hot_matrix']

,1,2,3,4,5,6,7,8,9,10,11,12,13
31.0,1,0,0,1,1,1,1,0,0,0,0,0,0
32.0,1,0,0,1,1,0,0,1,0,0,0,0,0
33.0,1,1,0,0,1,0,0,0,1,0,0,0,0
34.0,1,1,1,0,0,0,0,0,0,1,1,0,0
36.0,1,1,1,1,0,0,0,0,0,0,0,0,0
37.0,0,1,1,1,0,0,0,0,0,0,0,1,0
38.0,1,1,1,1,0,0,0,0,0,0,0,0,0
39.0,1,1,1,0,0,0,0,0,0,0,0,0,1
40.0,0,1,1,1,0,0,0,0,0,0,0,0,0
128.0,1,1,1,0,0,0,0,0,0,0,0,0,0


In [118]:
import os
import pickle
from processing_codenet import load_dkt_dataset, KTDataset, pad_collate, preprocess_train_dataset
saved_data_dir = "../data/492-2"
dataset_info = load_dataset_from_pkl(os.path.join(saved_data_dir, 'dataset_info.pkl'))
dataset_info['one_hot_matrix'] = dataset_info['one_hot_matrix'].to_numpy()
type(dataset_info['one_hot_matrix'])

numpy.ndarray

In [120]:
print(dataset_info['one_hot_matrix'].shape[0] == dataset_info['qt_num'])
print(dataset_info['one_hot_matrix'].shape[1] == dataset_info['concept_num'])

True
True


In [121]:
print(dataset_info['one_hot_matrix'].shape[0] == dataset_info['qt_num'])
print(dataset_info['one_hot_matrix'].shape[1] == dataset_info['concept_num'])
with open(os.path.join(saved_data_dir, 'dataset_info.pkl'), 'wb') as f:
    pickle.dump(dataset_info, f)

True
True
